<a href="https://colab.research.google.com/github/j-buss/wi-dpi-analysis/blob/development/eda/2.0_Landing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Salary and Education in Wisconsin

This notebook is intended to describe analysis on salaries of teachers within the Wisconsin Department of Public Instruction.

## Introduction

### Load libraries
Install the following packages in order to load data to BigQuery.

*Please note this will require a restart to the runtime*

In [0]:
!pip install gcsfs
!pip install pandas-gbq -U
import gcsfs

In [0]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 5)
import seaborn as sns
import matplotlib.pyplot as plt

from google.cloud import bigquery

In [0]:
%matplotlib inline
plt.style.use('bmh')

### Functions

In [0]:
def create_dataset(client, project_id, dataset_name):
  
  
  dataset_id = "{}.{}".format(project_id, dataset_name)
  dataset = bigquery.Dataset(dataset_id)
  dataset.location = "US"

  dataset = client.create_dataset(dataset)
  print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

### Define Values

In [0]:
project_id='wi-dpi-010'
raw_data_bucket_name='landing-009'
source_name='all_staff_report'
year='2017_2018'
filename='AllStaffReportPublic__04152019_194414.csv'
full_filename=raw_data_bucket_name + '/' + source_name + '/' + year + '/' + filename

landing_dataset_name='landing'
landing_table_name=source_name
landing_bq_fullname=landing_dataset_name + '.' + landing_table_name

refined_dataset_name='refined'
refined_table_name=source_name
refined_bq_fullname=refined_dataset_name + '.' + refined_table_name

## Data Preparation

### Create Dataset

### Create Tables

In [0]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

### Load Data

In [0]:
client = bigquery.Client(project_id)

**Load data from Google Cloud Storage Bucket to BigQuery Landing**

In [0]:
create_dataset(client, project_id, landing_dataset_name)

In [0]:
print (full_filename)

landing-009/all_staff_report/2017_2018/AllStaffReportPublic__04152019_194414.csv


In [0]:
fs = gcsfs.GCSFileSystem(project=project_id)


In [0]:
i = file_dict[0]

In [0]:
storage_client = storage.Client(project=project_id)
bucket = storage_client.get_bucket(raw_data_bucket_name)
blob = bucket.get_blob(i['new_name'])
data = blob.download_as_string()

In [0]:
print (data[0:1000])

b'000016044WILSON              JUDITH          FW1962419851995R                                    01 000000003300009888409125264N0802695H  001    NON-EDUCATIONAL AGENCY        NON-EDUCATIONAL AGENCY                                           33                                                                                                                                                                                                     WI                           WI\r\n000003685STARK               LISA                       1995R                                    01 00000070030999630003E120464N1000070OT 001    ALGOMA SCH DIST               ALGOMA SCH DIST               00731KEWAUNEE COUNTY               031715 DIVISION ST              ALGOMA WI  54201                                            1715 DIVISION ST              ALGOMA WI  54201                                            ALGOMA           WI54201     ALGOMA           WI54201     414-487-7001000780DALE N LARSON\r\n000014696

In [0]:
df = pd.read_fwf(data, colspecs='infer')

TypeError: ignored

In [0]:
fn = raw_data_bucket_name + '/' + i['new_name']
print(fn)

landing-009/all_staff_report/1995_1996/95staff.txt


In [0]:
with fs.open(fn) as f:
    #print(f.read())
    df = pd.read_fwf(f.read(), colspecs='infer')
    #df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    #df.to_gbq('landing.' + i['landing_tablename'],project_id=project_id,if_exists='replace')
  

TypeError: ignored

In [0]:
for i in file_dict:
  with fs.open(i['new_name']) as f:
    df = pd.read_csv(f, skiprows=1)
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df.to_gbq('landing.' + i['landing_tablename'],project_id=project_id,if_exists='replace')

FileNotFoundError: ignored

In [0]:

with fs.open(full_filename) as f:
  df = pd.read_csv(f, skiprows=1)

In [0]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df.to_gbq(landing_bq_fullname,project_id=project_id,if_exists='replace')